## Preprocessing

In [69]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU, Dropout

# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [70]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN', 'NAME'])

In [71]:
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [72]:
# Determine the number of unique values in each column- we are doing this in order to see what columns to use as features
application_df.nunique()

,0
APPLICATION_TYPE,17
AFFILIATION,6
CLASSIFICATION,71
USE_CASE,5
ORGANIZATION,4
STATUS,2
INCOME_AMT,9
SPECIAL_CONSIDERATIONS,2
ASK_AMT,8747
IS_SUCCESSFUL,2


In [73]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_df['APPLICATION_TYPE'].value_counts()

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
T9,156


In [74]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
Other,276


In [75]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
application_df['CLASSIFICATION'].value_counts()

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
...,...
C1248,1
C6100,1
C1820,1


In [76]:
greater_than_1 = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() <= 1]
greater_than_1.index.to_list()


['C2561',
 'C1245',
 'C4500',
 'C1732',
 'C1570',
 'C2380',
 'C4120',
 'C2170',
 'C1728',
 'C8210',
 'C2570',
 'C1580',
 'C3700',
 'C2500',
 'C1283',
 'C1236',
 'C4200',
 'C2190',
 'C5200',
 'C1370',
 'C2600',
 'C1248',
 'C6100',
 'C1820',
 'C1900',
 'C2150']

In [12]:
# APPLICATION_TYPE	17
# AFFILIATION	6
# CLASSIFICATION	71
# USE_CASE	5
# ORGANIZATION	4
# STATUS	2
# INCOME_AMT	9
# SPECIAL_CONSIDERATIONS	2
# ASK_AMT	8747
# IS_SUCCESSFUL	2


,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,5000,1,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,108590,1,False,False,False,True,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,5000,0,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False
3,6692,1,False,False,False,True,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,142590,1,False,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,5000,0,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
34295,5000,0,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
34296,5000,0,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
34297,5000,1,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False


In [77]:
# You may find it helpful to look at CLASSIFICATION value counts >1
# application_df['CLASSIFICATION'].value_counts().loc[lambda x : x>1]- simpler function where it uses a function to make it
application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() > 1]


,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
C7000,777
C1700,287
C4000,194
C5000,116


In [78]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = ['C2800', 'C7100', 'C1300', 'C1280', 'C1230', 'C1400', 'C7200', 'C2300', 'C1240', 'C8000', 'C7120',
'C1500', 'C6000', 'C1800', 'C1250', 'C8200', 'C1238', 'C1278', 'C1237', 'C1235', 'C7210', 'C1720', 'C2400', 'C4100', 'C1257',
'C1600', 'C1260', 'C2710', 'C0', 'C3200', 'C1256', 'C1246', 'C1234', 'C1267', 'C7000', 'C1700', 'C4000', 'C5000', 'C1270', 'C2700', 'C2561',
'C1245', 'C4500', 'C8210', 'C4120', 'C2170', 'C1728', 'C1732', 'C2380', 'C1283', 'C1570', 'C2500', 'C3700', 'C1580', 'C2570', 'C1236','C2190',
'C4200', 'C5200', 'C1370', 'C2600', 'C1248', 'C6100', 'C1820', 'C1900', 'C2150']

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
Other,2261
C3000,1918
C2100,1883


In [95]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies = pd.get_dummies(application_df, columns=['APPLICATION_TYPE', 'CLASSIFICATION', 'AFFILIATION', 'ORGANIZATION', 'USE_CASE', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'])
dummies

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,5000,1,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,108590,1,False,False,False,True,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,5000,0,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False
3,6692,1,False,False,False,True,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,142590,1,False,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,5000,0,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
34295,5000,0,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
34296,5000,0,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
34297,5000,1,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [96]:
# Split our preprocessed data into our features and target arrays
y = dummies['IS_SUCCESSFUL']
X = dummies.drop(columns=['IS_SUCCESSFUL'], axis=1)


# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [97]:
# Create a StandardScaler instances
scaler = StandardScaler()
scaler.fit(X_train)

# Fit the StandardScaler
# X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [98]:
#I want to see if there are more 1s than 0s
y_train.value_counts(normalize=True)


,proportion
IS_SUCCESSFUL,
1,0.532149
0,0.467851


In [99]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))


## Compile, Train and Evaluate the Model

In [100]:
# #THE BELOW CODE I TRIED 3 LAYERS, I MAY NOT NEED THREE LAYERS, I MIGHT HAVE JUST NEEDED TO INCREASE THE UNITS
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=160, activation="relu", input_dim=44))
nn.add(tf.keras.layers.BatchNormalization())
nn.add(tf.keras.layers.Dropout(0.2))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu"))
nn.add(tf.keras.layers.Dropout(0.1))

# Third hidden layer- added this third layer and increased the untis in order to be able to increase the accuracy of the model, I am currently stuck 73
nn.add(tf.keras.layers.Dense(units=40, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_61 (Dense)                │ (None, 160)            │         7,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 160)            │           640 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 160)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_62 (Dense)                │ (None, 80)             │        12,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_23 (Dropout)            │ (None, 80)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_63 (Dense)                │ (None, 40)             │         3,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_64 (Dense)                │ (None, 1)              │            41 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,001 (93.75 KB)

 Trainable params: 23,681 (92.50 KB)

 Non-trainable params: 320 (1.25 KB)

In [56]:
# nn = tf.keras.models.Sequential()
# #LAYER 1
# nn.add(tf.keras.layers.Dense(units=128, activation="relu", input_dim=109))
# #LAYER 2
# nn.add(tf.keras.layers.Dense(units=64, activation="relu"))
# #OUTPUT LAYER
# nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# # Check the structure of the model
# nn.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_34 (Dense)                │ (None, 128)            │        14,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,401 (87.50 KB)

 Trainable params: 22,401 (87.50 KB)

 Non-trainable params: 0 (0.00 B)

In [49]:
#trying LeakyRelu instead of Relu- DID NOT IMPROVE ACCURACY from regular RELU
# nn = Sequential()
# nn.add(Dense(128, input_dim=109))
# nn.add(LeakyReLU(alpha=0.1))
# nn.add(Dropout(0.2))

# nn.add(Dense(64))
# nn.add(LeakyReLU(alpha=0.1))
# nn.add(Dropout(0.1))

# nn.add(Dense(1, activation='sigmoid'))

# nn.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_31 (Dense)                │ (None, 128)            │        14,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_32 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,401 (87.50 KB)

 Trainable params: 22,401 (87.50 KB)

 Non-trainable params: 0 (0.00 B)

In [50]:
#TESTING LEAKY RELU- DID NOT WORK
# from tensorflow.keras.callbacks import ReduceLROnPlateau

# lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)


In [101]:
# # Compile the model
# nn.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
# #fit the model
# fit_model = nn.fit(X_train_scaled, y_train, epochs=50, validation_split=0.2, class_weight=class_weight_dict, verbose=2)

#importing adam but adjusting it

from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate= 0.0005)

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

fit_model = nn.fit(X_train_scaled, y_train,
                   epochs=100,
                   validation_split=0.2,
                   callbacks=[early_stop],
                   verbose=2)


Epoch 1/100
644/644 - 5s - 7ms/step - accuracy: 0.7102 - loss: 0.5875 - val_accuracy: 0.7180 - val_loss: 0.5720
Epoch 2/100
644/644 - 4s - 6ms/step - accuracy: 0.7253 - loss: 0.5648 - val_accuracy: 0.7217 - val_loss: 0.5680
Epoch 3/100
644/644 - 4s - 6ms/step - accuracy: 0.7272 - loss: 0.5607 - val_accuracy: 0.7234 - val_loss: 0.5664
Epoch 4/100
644/644 - 3s - 4ms/step - accuracy: 0.7299 - loss: 0.5570 - val_accuracy: 0.7240 - val_loss: 0.5665
Epoch 5/100
644/644 - 5s - 8ms/step - accuracy: 0.7326 - loss: 0.5553 - val_accuracy: 0.7195 - val_loss: 0.5672
Epoch 6/100
644/644 - 5s - 8ms/step - accuracy: 0.7322 - loss: 0.5557 - val_accuracy: 0.7238 - val_loss: 0.5616
Epoch 7/100
644/644 - 5s - 8ms/step - accuracy: 0.7343 - loss: 0.5520 - val_accuracy: 0.7230 - val_loss: 0.5627
Epoch 8/100
644/644 - 5s - 8ms/step - accuracy: 0.7356 - loss: 0.5515 - val_accuracy: 0.7209 - val_loss: 0.5653
Epoch 9/100
644/644 - 5s - 7ms/step - accuracy: 0.7327 - loss: 0.5519 - val_accuracy: 0.7242 - val_loss:

In [22]:
# from sklearn.utils.class_weight import compute_class_weight
# import numpy as np
# # Compile the model
# nn.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

# #added code

# class_weights = compute_class_weight(class_weight='balanced',
#                                      classes=np.unique(y_train),
#                                      y=y_train)

# class_weight_dict = dict(enumerate(class_weights))

# #fit the model
# early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
# fit_model = nn.fit(X_train_scaled, y_train, epochs=50, validation_split=0.2, callbacks=[early_stop], class_weight=class_weight_dict, verbose=2)

Epoch 1/50


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_7" is incompatible with the layer: expected axis -1 of input shape to have value 44, but received input with shape (None, 109)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 109), dtype=float32)
  • training=True
  • mask=None

In [64]:
# #BALANCING THE WEIGHTS OF THE 0s and 1s
# # Optional: compute class weights
# class_weights = compute_class_weight(class_weight='balanced',
#                                      classes=np.unique(y_train),
#                                      y=y_train)
# class_weight_dict = dict(enumerate(class_weights))

# # Define model
# nn = Sequential()
# nn.add(Dense(128, activation='relu', input_dim=X_train_scaled.shape[1]))
# nn.add(BatchNormalization())
# nn.add(Dropout(0.3))

# nn.add(Dense(64, activation='relu'))
# nn.add(Dropout(0.2))

# nn.add(Dense(32, activation='relu'))

# nn.add(Dense(1, activation='sigmoid'))

# # Compile
# nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# # Train
# early_stop = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)

# fit_model = nn.fit(X_train_scaled, y_train,
#                    validation_split=0.2,
#                    epochs=100,
#                    callbacks=[early_stop],
#                    class_weight=class_weight_dict,
#                    verbose=2)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
644/644 - 7s - 11ms/step - accuracy: 0.7028 - loss: 0.5940 - val_accuracy: 0.7207 - val_loss: 0.5740
Epoch 2/100
644/644 - 5s - 8ms/step - accuracy: 0.7230 - loss: 0.5719 - val_accuracy: 0.7219 - val_loss: 0.5756
Epoch 3/100
644/644 - 3s - 5ms/step - accuracy: 0.7277 - loss: 0.5645 - val_accuracy: 0.7258 - val_loss: 0.5731
Epoch 4/100
644/644 - 4s - 7ms/step - accuracy: 0.7302 - loss: 0.5625 - val_accuracy: 0.7244 - val_loss: 0.5713
Epoch 5/100
644/644 - 3s - 4ms/step - accuracy: 0.7327 - loss: 0.5579 - val_accuracy: 0.7256 - val_loss: 0.5648
Epoch 6/100
644/644 - 6s - 9ms/step - accuracy: 0.7325 - loss: 0.5566 - val_accuracy: 0.7234 - val_loss: 0.5697
Epoch 7/100
644/644 - 4s - 7ms/step - accuracy: 0.7332 - loss: 0.5570 - val_accuracy: 0.7263 - val_loss: 0.5624
Epoch 8/100
644/644 - 3s - 4ms/step - accuracy: 0.7333 - loss: 0.5540 - val_accuracy: 0.7250 - val_loss: 0.5689
Epoch 9/100
644/644 - 3s - 5ms/step - accuracy: 0.7308 - loss: 0.5531 - val_accuracy: 0.7258 - val_loss

In [102]:
from sklearn.metrics import classification_report, confusion_matrix
y_pred = (nn.predict(X_test_scaled) > 0.5).astype("int32")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

268/268 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
[[2705 1298]
 [1022 3550]]
              precision    recall  f1-score   support

           0       0.73      0.68      0.70      4003
           1       0.73      0.78      0.75      4572

    accuracy                           0.73      8575
   macro avg       0.73      0.73      0.73      8575
weighted avg       0.73      0.73      0.73      8575



In [103]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - 2ms/step - accuracy: 0.7294 - loss: 0.5524
Loss: 0.5524042248725891, Accuracy: 0.7294460535049438


In [104]:
# Export our model to HDF5 file
nn.save("charity_model.h5")